In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
pd.options.display.float_format = '{:.2f}'.format
import pandas as pd
import numpy as np
%matplotlib inline
plt.rcParams["axes.formatter.limits"] = (-5, 12)
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
from scipy import stats
import shap

/home/stnog/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
Dados_LDL = pd.read_excel('/home/stnog/Modelo_Boleto/Dados_LDL.xlsx')
Dados_Boleto_17h = pd.read_excel('/home/stnog/Modelo_Boleto/Dados_Boleto_17h.xlsx')
Contas_ativas = pd.read_excel('/home/stnog/Modelo_Boleto/Contas_Ativas.xlsx')
Dados_LDL

,inserted_at,valor_brl,code
0,2022-08-12 10:01:17.271,27946546.34,LDL0021
1,2022-08-11 10:00:59.414,38249311.84,LDL0021
2,2022-08-10 10:00:59.290,3289424.40,LDL0021
3,2022-08-09 10:00:45.688,11225517.79,LDL0021
4,2022-08-08 10:01:22.434,16430459.44,LDL0021
...,...,...,...
405,2021-01-08 10:00:53.339,10678826.92,LDL0021
406,2021-01-07 10:01:03.091,11441470.87,LDL0021
407,2021-01-06 10:00:31.806,15693545.34,LDL0021
408,2021-01-05 10:00:46.254,47522788.37,LDL0021


In [54]:
feriados = pd.read_csv('/home/stnog/Modelo_Boleto/Feriados.csv')
feriado=pd.to_datetime(feriados['Data'])
feriados=pd.to_datetime(feriados['Data']).astype('str').tolist()
dia_de_semana = pd.bdate_range('2021-01-01', '2037-12-31')
dia_de_semana = dia_de_semana.astype('str').tolist()
wDays = [x for x in dia_de_semana if x not in feriados ]

In [55]:
Contas_ativas = Contas_ativas.rename(columns={'dia': 'Dia'})
Contas_ativas

,Dia,contas_ativas
0,2021-01-01,9861
1,2021-01-02,16748
2,2021-01-03,12766
3,2021-01-04,93248
4,2021-01-05,82573
...,...,...
587,2022-08-11,329027
588,2022-08-12,337290
589,2022-08-13,210638
590,2022-08-14,140831


In [56]:
Dados_Boleto_17h = Dados_Boleto_17h.rename(columns={'paid_at': 'Dia'})
Dados_Boleto_17h

,Dia,sum,count
0,2021-01-01,777769.44,1134
1,2021-01-02,1688196.95,3303
2,2021-01-03,1149937.54,1904
3,2021-01-04,30095294.37,35960
4,2021-01-05,12722553.82,18050
...,...,...,...
587,2022-08-12,28267553.43,37632
588,2022-08-13,7228940.96,11852
589,2022-08-14,2095309.70,3104
590,2022-08-15,1473718.02,1488


In [57]:
Dados_Boleto_17h['weekday'] = pd.to_datetime(Dados_Boleto_17h['Dia']).dt.weekday
Dados_Boleto_17h

,Dia,sum,count,weekday
0,2021-01-01,777769.44,1134,4
1,2021-01-02,1688196.95,3303,5
2,2021-01-03,1149937.54,1904,6
3,2021-01-04,30095294.37,35960,0
4,2021-01-05,12722553.82,18050,1
...,...,...,...,...
587,2022-08-12,28267553.43,37632,4
588,2022-08-13,7228940.96,11852,5
589,2022-08-14,2095309.70,3104,6
590,2022-08-15,1473718.02,1488,0


In [58]:
Dados_Boleto_17h['ajuste_domingo_sum'] = Dados_Boleto_17h['sum'].shift(1)
Dados_Boleto_17h['ajuste_domingo_count'] = Dados_Boleto_17h['count'].shift(1)
Dados_Boleto_17h['ajuste_sab_sum'] = Dados_Boleto_17h['sum'].shift(2)
Dados_Boleto_17h['ajuste_sab_count'] = Dados_Boleto_17h['count'].shift(2)
Dados_Boleto_17h

,Dia,sum,count,weekday,ajuste_domingo_sum,ajuste_domingo_count,ajuste_sab_sum,ajuste_sab_count
0,2021-01-01,777769.44,1134,4,nan,nan,nan,nan
1,2021-01-02,1688196.95,3303,5,777769.44,1134.00,nan,nan
2,2021-01-03,1149937.54,1904,6,1688196.95,3303.00,777769.44,1134.00
3,2021-01-04,30095294.37,35960,0,1149937.54,1904.00,1688196.95,3303.00
4,2021-01-05,12722553.82,18050,1,30095294.37,35960.00,1149937.54,1904.00
...,...,...,...,...,...,...,...,...
587,2022-08-12,28267553.43,37632,4,19428825.86,26589.00,25596381.09,32553.00
588,2022-08-13,7228940.96,11852,5,28267553.43,37632.00,19428825.86,26589.00
589,2022-08-14,2095309.70,3104,6,7228940.96,11852.00,28267553.43,37632.00
590,2022-08-15,1473718.02,1488,0,2095309.70,3104.00,7228940.96,11852.00


In [59]:
Dados_Boleto_17h['valor'] = np.where(Dados_Boleto_17h['weekday']==0, Dados_Boleto_17h['sum']+Dados_Boleto_17h['ajuste_domingo_sum']+Dados_Boleto_17h['ajuste_sab_sum'], Dados_Boleto_17h['sum'])
Dados_Boleto_17h['Qtd'] = np.where(Dados_Boleto_17h['weekday']==0, Dados_Boleto_17h['count']+Dados_Boleto_17h['ajuste_domingo_count']+Dados_Boleto_17h['ajuste_sab_count'], Dados_Boleto_17h['count'])
Dados_Boleto_17h

,Dia,sum,count,weekday,ajuste_domingo_sum,ajuste_domingo_count,ajuste_sab_sum,ajuste_sab_count,valor,Qtd
0,2021-01-01,777769.44,1134,4,nan,nan,nan,nan,777769.44,1134.00
1,2021-01-02,1688196.95,3303,5,777769.44,1134.00,nan,nan,1688196.95,3303.00
2,2021-01-03,1149937.54,1904,6,1688196.95,3303.00,777769.44,1134.00,1149937.54,1904.00
3,2021-01-04,30095294.37,35960,0,1149937.54,1904.00,1688196.95,3303.00,32933428.86,41167.00
4,2021-01-05,12722553.82,18050,1,30095294.37,35960.00,1149937.54,1904.00,12722553.82,18050.00
...,...,...,...,...,...,...,...,...,...,...
587,2022-08-12,28267553.43,37632,4,19428825.86,26589.00,25596381.09,32553.00,28267553.43,37632.00
588,2022-08-13,7228940.96,11852,5,28267553.43,37632.00,19428825.86,26589.00,7228940.96,11852.00
589,2022-08-14,2095309.70,3104,6,7228940.96,11852.00,28267553.43,37632.00,2095309.70,3104.00
590,2022-08-15,1473718.02,1488,0,2095309.70,3104.00,7228940.96,11852.00,10797968.68,16444.00


In [60]:
Colunas_rem = ['sum','count','ajuste_domingo_sum','ajuste_domingo_count','ajuste_sab_sum','ajuste_sab_count']
Dados_Boleto_17h = Dados_Boleto_17h.drop(Colunas_rem, axis=1)
Dados_Boleto_17h

,Dia,weekday,valor,Qtd
0,2021-01-01,4,777769.44,1134.00
1,2021-01-02,5,1688196.95,3303.00
2,2021-01-03,6,1149937.54,1904.00
3,2021-01-04,0,32933428.86,41167.00
4,2021-01-05,1,12722553.82,18050.00
...,...,...,...,...
587,2022-08-12,4,28267553.43,37632.00
588,2022-08-13,5,7228940.96,11852.00
589,2022-08-14,6,2095309.70,3104.00
590,2022-08-15,0,10797968.68,16444.00


In [61]:
Dados_Boleto_17h = Dados_Boleto_17h[(Dados_Boleto_17h['weekday']!=6)&(Dados_Boleto_17h['weekday']!=5)].reset_index(drop=True)
Dados_Boleto_17h

,Dia,weekday,valor,Qtd
0,2021-01-01,4,777769.44,1134.00
1,2021-01-04,0,32933428.86,41167.00
2,2021-01-05,1,12722553.82,18050.00
3,2021-01-06,2,9358715.15,14317.00
4,2021-01-07,3,9006199.89,13511.00
...,...,...,...,...
417,2022-08-10,2,25596381.09,32553.00
418,2022-08-11,3,19428825.86,26589.00
419,2022-08-12,4,28267553.43,37632.00
420,2022-08-15,0,10797968.68,16444.00


In [62]:
Dados_Boleto_17h['feriado'] = np.where(Dados_Boleto_17h.Dia.isin(pd.to_datetime(feriados)), 1, 0)

In [63]:
Dados_Boleto_17h = Dados_Boleto_17h.sort_values('Dia')
Dados_Boleto_17h

,Dia,weekday,valor,Qtd,feriado
0,2021-01-01,4,777769.44,1134.00,1
1,2021-01-04,0,32933428.86,41167.00,0
2,2021-01-05,1,12722553.82,18050.00,0
3,2021-01-06,2,9358715.15,14317.00,0
4,2021-01-07,3,9006199.89,13511.00,0
...,...,...,...,...,...
417,2022-08-10,2,25596381.09,32553.00,0
418,2022-08-11,3,19428825.86,26589.00,0
419,2022-08-12,4,28267553.43,37632.00,0
420,2022-08-15,0,10797968.68,16444.00,0


In [73]:
def feriado(Dados_Boleto_17h):
    for i in Dados_Boleto_17h.index:
        if Dados_Boleto_17h['feriado'].loc[i] == 1:
            Dados_Boleto_17h['valor'].loc[i+1] = Dados_Boleto_17h['valor'].loc[i+1] + Dados_Boleto_17h['valor'].loc[i]
            Dados_Boleto_17h['Qtd'].loc[i+1] = Dados_Boleto_17h['Qtd'].loc[i+1] + Dados_Boleto_17h['Qtd'].loc[i]
    return Dados_Boleto_17h

In [74]:
feriado(Dados_Boleto_17h)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Dia,weekday,valor,Qtd,feriado
0,2021-01-01,4,777769.44,1134.00,1
1,2021-01-04,0,33711198.30,42301.00,0
2,2021-01-05,1,12722553.82,18050.00,0
3,2021-01-06,2,9358715.15,14317.00,0
4,2021-01-07,3,9006199.89,13511.00,0
...,...,...,...,...,...
417,2022-08-10,2,25596381.09,32553.00,0
418,2022-08-11,3,19428825.86,26589.00,0
419,2022-08-12,4,28267553.43,37632.00,0
420,2022-08-15,0,10797968.68,16444.00,0


In [76]:
Dados_Boleto_17h = Dados_Boleto_17h[Dados_Boleto_17h['feriado'] != 1].reset_index(drop=True)

In [79]:
Dados_Boleto_17h['Previa_LDL_D-1'] = Dados_Boleto_17h['valor'].shift(1)
Dados_Boleto_17h['Qtd_Bol_Previa_D-1'] = Dados_Boleto_17h['Qtd'].shift(1)

In [80]:
Dados_Boleto_17h

,Dia,weekday,valor,Qtd,feriado,Previa_LDL_D-1,Qtd_Bol_Previa_D-1
0,2021-01-04,0,33711198.30,42301.00,0,nan,nan
1,2021-01-05,1,12722553.82,18050.00,0,33711198.30,42301.00
2,2021-01-06,2,9358715.15,14317.00,0,12722553.82,18050.00
3,2021-01-07,3,9006199.89,13511.00,0,9358715.15,14317.00
4,2021-01-08,4,8665800.74,12637.00,0,9006199.89,13511.00
...,...,...,...,...,...,...,...
402,2022-08-10,2,25596381.09,32553.00,0,20755232.26,27057.00
403,2022-08-11,3,19428825.86,26589.00,0,25596381.09,32553.00
404,2022-08-12,4,28267553.43,37632.00,0,19428825.86,26589.00
405,2022-08-15,0,10797968.68,16444.00,0,28267553.43,37632.00


In [6]:
Dados_LDL['Dia'] = pd.to_datetime(Dados_LDL['inserted_at']).dt.date
Dados_LDL = Dados_LDL.drop('inserted_at', axis=1)
Dados_LDL

,valor_brl,code,Dia
0,27946546.34,LDL0021,2022-08-12
1,38249311.84,LDL0021,2022-08-11
2,3289424.40,LDL0021,2022-08-10
3,11225517.79,LDL0021,2022-08-09
4,16430459.44,LDL0021,2022-08-08
...,...,...,...
405,10678826.92,LDL0021,2021-01-08
406,11441470.87,LDL0021,2021-01-07
407,15693545.34,LDL0021,2021-01-06
408,47522788.37,LDL0021,2021-01-05


In [7]:
Dados_LDL = Dados_LDL[Dados_LDL['valor_brl']!=0].reset_index(drop=True)
Dados_LDL

,valor_brl,code,Dia
0,27946546.34,LDL0021,2022-08-12
1,38249311.84,LDL0021,2022-08-11
2,3289424.40,LDL0021,2022-08-10
3,11225517.79,LDL0021,2022-08-09
4,16430459.44,LDL0021,2022-08-08
...,...,...,...
398,10233998.77,LDL0021,2021-01-11
399,10678826.92,LDL0021,2021-01-08
400,11441470.87,LDL0021,2021-01-07
401,15693545.34,LDL0021,2021-01-06


In [22]:
feriados = pd.read_csv('/home/stnog/Modelo_Boleto/Feriados.csv')
feriado=pd.to_datetime(feriados['Data'])
feriados=pd.to_datetime(feriados['Data']).astype('str').tolist()
dia_de_semana = pd.bdate_range('2021-01-01', '2037-12-31')
dia_de_semana = dia_de_semana.astype('str').tolist()
wDays = [x for x in dia_de_semana if x not in feriados ]